### 커리어넷 API KEY를 사용하여 학과주요진출분야 가져오기

커리어넷 API를 활용하여 입력된 학과명을 기반으로
- 대표학과명 매칭
- 학과 상세 정보(계열, 관련학과, 관련직업, 자격, 졸업 후 진출 분야) 수집
- 매칭 여부(정확 일치 / 관련 학과 포함 / 미매칭) 판단
- '졸업 후 진출 분야' 항목을 정규식 기반으로 기업/학계/공공기관으로 분리
- 전체 결과를 pandas DataFrame으로 구성한 후, 엑셀 파일로 저장합니다.

[주요 구성]
1. 학과명 전처리 (기호 제거 및 공백 정리)
2. 커리어넷 API를 통해 전체 학과 리스트 수집
3. 학과 코드 기반 상세 정보 수집 (직업, 자격, 진출 분야 등)
4. 입력 학과명과의 매칭 (대표명 정확 일치 → 관련학과 포함 순)
5. 졸업 후 진출 분야를 정규식 기반으로 분리 (기업 / 학계 / 공공기관)
6. 결과 DataFrame 생성 및 엑셀 저장

※ API Key 필요: 커리어넷 Open API 등록 필요
※ 입력 엑셀 파일: '학과기준.xlsx' 파일에서 '학과명' 컬럼 사용

### 커리어넷에서 필요 정보 불러오기

In [ ]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import re

#기준이 될 학과명이 들어간 파일 불러오기
status_file = r"C:\Users\윤서현\Desktop\국립강릉원주대 AI STUDIO\학과기준.xlsx"
status_df = pd.read_excel(status_file)

#학과명 정리 함수
def clean_major_name(df, col):
    df[col] = df[col].str.replace('·', '', regex=False)
    df[col] = df[col].str.replace('・', '', regex=False)
    df[col] = df[col].str.replace('?', '', regex=False)
    df[col] = df[col].str.strip()
    return df

status_df_clean = clean_major_name(status_df, '학과명')
# 학과 리스트 추출
target_majors = status_df_clean['학과명'].tolist()



# 학과 리스트 수집 함수
def fetch_major_list(api_key):
    url = 'https://www.career.go.kr/cnet/openapi/getOpenApi'
    page = 1
    per_page = 100
    total_count = 0
    major_list = []

    while True:
        params = {
            'apiKey': api_key,
            'svcType': 'api',
            'svcCode': 'MAJOR',
            'contentType': 'xml',
            'gubun': 'univ_list',
            'thisPage': str(page),
            'perPage': str(per_page)
        }

        response = requests.get(url, params=params)
        response.raise_for_status()

        root = ET.fromstring(response.content)
        contents = root.findall('.//content')

        if not contents:
            print(f"모든 페이지를 불러왔습니다. 총 {total_count}건 수집 완료.")
            break

        for idx, content in enumerate(contents, 1):
            l_class = content.findtext('lClass')
            m_class = content.findtext('mClass')
            major_seq = content.findtext('majorSeq')
            facil_name = content.findtext('facilName')

            total_count += 1

            major_list.append({
                '계열': l_class,
                '대표학과명': m_class,
                '학과코드': major_seq,
                '관련학과': facil_name
            })

        page += 1

    return major_list


# 학과 상세 정보 조회 함수
def fetch_major_detail(api_key, major_seq):
    url = 'https://www.career.go.kr/cnet/openapi/getOpenApi'
    params = {
        'apiKey': api_key,
        'svcType': 'api',
        'svcCode': 'MAJOR_VIEW',
        'contentType': 'xml',
        'gubun': 'univ_list',
        'majorSeq': str(major_seq)
    }

    response = requests.get(url, params=params)
    if response.status_code == 200:
        root = ET.fromstring(response.content)

        job = root.findtext('.//job')
        qualifications = root.findtext('.//qualifications')
        enter_fields = root.findall('.//enter_field/content')

        detail_info = {
            '관련 직업': job if job else '없음',
            '관련 자격': qualifications if qualifications else '없음',
            '졸업 후 진출 분야': []
        }

        if enter_fields:
            for field in enter_fields:
                gradeuate = field.findtext('gradeuate')
                description = field.findtext('description')
                detail_info['졸업 후 진출 분야'].append(f"{gradeuate} - {description}")

        return detail_info

    else:
        print(f'Error: {response.status_code}')
        return None


# 학과 상세 정보 출력 함수
def fetch_and_display_detail(api_key, major_seq):
    detail = fetch_major_detail(api_key, major_seq)
    if detail:
        print(f"   관련 직업: {detail['관련 직업']}")
        print(f"   관련 자격: {detail['관련 자격']}")
        print(f"   졸업 후 진출 분야:")
        if detail['졸업 후 진출 분야']:
            for field in detail['졸업 후 진출 분야']:
                print(f"    - {field}")
        else:
            print(f"    없음")


# 스마트 검색 (정확 일치 > 관련 학과 포함)
def search_major(api_key, major_list, search_name, verbose=False):
    result = {
        '입력 학과명': search_name.strip(),
        '매칭된 대표학과명': None,
        '학과코드': None,
        '계열': None,
        '관련학과': None,
        '관련 직업': None,
        '관련 자격': None,
        '졸업 후 진출 분야': None,
        '매칭 방식': None
    }

    exact_matches = [m for m in major_list if search_name == m['대표학과명']]
    if exact_matches:
        major = exact_matches[0]
        detail = fetch_major_detail(api_key, major['학과코드'])
        result.update({
            '매칭된 대표학과명': major['대표학과명'],
            '학과코드': major['학과코드'],
            '계열': major['계열'],
            '관련학과': major['관련학과'],
            '관련 직업': detail['관련 직업'] if detail else None,
            '관련 자격': detail['관련 자격'] if detail else None,
            '졸업 후 진출 분야': "\n".join(detail['졸업 후 진출 분야']) if detail else None,
            '매칭 방식': '정확 일치'
        })
        if verbose:
            print(f"[정확 일치] {search_name} → {major['대표학과명']}")

    else:
        related_matches = [m for m in major_list if m['관련학과'] and search_name in m['관련학과']]
        if related_matches:
            major = related_matches[0]
            detail = fetch_major_detail(api_key, major['학과코드'])
            result.update({
                '매칭된 대표학과명': major['대표학과명'],
                '학과코드': major['학과코드'],
                '계열': major['계열'],
                '관련학과': major['관련학과'],
                '관련 직업': detail['관련 직업'] if detail else None,
                '관련 자격': detail['관련 자격'] if detail else None,
                '졸업 후 진출 분야': "\n".join(detail['졸업 후 진출 분야']) if detail else None,
                '매칭 방식': '관련 학과 포함'
            })
            if verbose:
                print(f"[관련 학과 포함] {search_name} → {major['대표학과명']}")
        else:
            result['매칭 방식'] = '미매칭'
            if verbose:
                print(f"[미매칭] {search_name} → 결과 없음")

    return result



#결과 수집 및 저장
results = []
for name in target_majors:
    result = search_major(API_KEY, majors, name, verbose=True)
    results.append(result)

df = pd.DataFrame(results)


[정확 일치] 국어국문학과 → 국어국문학과
[정확 일치] 영어영문학과 → 영어영문학과
[정확 일치] 중어중문학과 → 중어중문학과
[정확 일치] 철학과 → 철학과
[정확 일치] 사학과 → 사학과
[정확 일치] 일본학과 → 일본학과
[정확 일치] 생물학과 → 생물학과
[정확 일치] 사회복지학과 → 사회복지학과
[정확 일치] 경영학과 → 경영학과
[정확 일치] 회계학과 → 회계학과
[정확 일치] 관광경영학과 → 관광경영학과
[정확 일치] 국제통상학과 → 국제통상학과
[관련 학과 포함] 도시계획부동산학과 → 도시계획학과
[정확 일치] 경제학과 → 경제학과
[정확 일치] 무역학과 → 무역학과
[관련 학과 포함] 자치행정학과 → 지방행정과
[미매칭] 법률정책학과 → 결과 없음
[관련 학과 포함] 수학물리학부 → 물리학과
[관련 학과 포함] 수학 → 농수산과
[미매칭] 물리.에너지 → 결과 없음
[관련 학과 포함] 대기환경과학과 → 대기과학과
[관련 학과 포함] 수산생명의학과 → 생명과학과
[정확 일치] 식품영양학과 → 식품영양학과
[관련 학과 포함] 식물생명과학과 → 생명과학과
[관련 학과 포함] 환경조경학과 → 조경과
[미매칭] 조형예술디자인학과 → 결과 없음
[관련 학과 포함] 조형예술 → 미술학과
[정확 일치] 디자인학과 → 디자인학과
[정확 일치] 체육학과 → 체육학과
[정확 일치] 음악학과 → 음악학과
[정확 일치] 패션디자인학과 → 패션디자인학과
[미매칭] 치의예과/치의학과 → 결과 없음
[정확 일치] 치위생학과 → 치위생학과
[정확 일치] 간호학과 → 간호학과
[관련 학과 포함] 다문화학과 → 교양인문학부
[정확 일치] 전기공학과 → 전기공학과
[미매칭] 디지털미디어커뮤니케이션학부 → 결과 없음
[정확 일치] 유아교육과 → 유아교육과
[미매칭] 스마트인프라공학부 → 결과 없음
[관련 학과 포함] 건설환경 → 건설정보과
[미매칭] 스마트공간정보 → 결과 없음
[미매칭] 전자반도체공학부 → 결과 없음
[관련 학과 포함] 반도체 → 물리학과
[관련 학과 포함] 전

### 졸업 후 진출 분야 열로 쪼개기

In [ ]:
# 정규식 기반 키워드 매핑 테이블 (표시명 -> 컬럼명)
keyword_to_column = {
    '기업 및 산업체': '졸업 후 진출 분야_기업 및 산업체',
    '학계 및 연구기관': '졸업 후 진출 분야_학계 및 연구기관',
    '정부 및 공공기관': '졸업 후 진출 분야_정부 및 공공기관',
    '정부 및 공공기관, 학교': '졸업 후 진출 분야_정부 및 공공기관'
}

# 정규식 분리용 패턴 (괄호 안은 캡처용)
pattern = r'(기업 및 산업체|학계 및 연구기관|정부 및 공공기관(?:,?\s*학교)?)\s*[-/:]?'

# 4. 전처리 함수
def extract_fields(text):
    result = {
        '졸업 후 진출 분야_기업 및 산업체': '',
        '졸업 후 진출 분야_학계 및 연구기관': '',
        '졸업 후 진출 분야_정부 및 공공기관': ''
    }
    if pd.isna(text):
        return result

    # 줄바꿈이나 슬래시 등 정리
    text = text.replace('\n', ' ').replace('/', '-')

    # 키워드로 분리
    parts = re.split(pattern, text)
    for key, value in zip(parts[1::2], parts[2::2]):
        key_clean = key.strip()
        colname = keyword_to_column.get(key_clean)
        if colname:
            result[colname] = value.strip()
    return result

# 적용
split_fields = df['졸업 후 진출 분야'].apply(extract_fields)
split_df = pd.DataFrame(split_fields.tolist())

# 병합
df_final = pd.concat([df.drop(columns=['졸업 후 진출 분야']), split_df], axis=1)

In [30]:
df_final.head()

,입력 학과명,매칭된 대표학과명,학과코드,계열,관련학과,관련 직업,관련 자격,매칭 방식,졸업 후 진출 분야_기업 및 산업체,졸업 후 진출 분야_학계 및 연구기관,졸업 후 진출 분야_정부 및 공공기관
0,국어국문학과,국어국문학과,69,인문계열,"Dept. of Global Korean Studies,KFL학부,KLC학과,Kor...","구성작가, 극작가, 기자, 네이미스트, 독서지도사, 방송연출가, 사서, 소설가, 시...","국어능력인증시험, 사서, 중등학교2급정교사, 한국어교육능력검정시험",정확 일치,"출판사, 광고 기획사, 광고 대행사, 기업 일반 사무직, 사설 학원, 신문사, 잡지...",언어ㆍ민족ㆍ문화 관련 국가ㆍ민간연구소,"한국교육학술정보원, 한국고전번역원, 한국언론진흥재단, 한국출판문화산업진흥원 등 언어..."
1,영어영문학과,영어영문학과,366,인문계열,"EICC학과,G2융합학과,TESOL영어학전공,관광영어통역전공,관광영어통역학전공,교양...","무역담당자, 언론인(기자,PD,아나운서등), 여행안내원, 인문과학연구원, 작가, 출...","관광통역안내사, 무역영어, 실용영어, 영어능력시험(Flex영어,TOEIC,TEPS,...",정확 일치,"출판사, 무역 회사, 여행사, 호텔, 기업 일반 사무직 및 해외 영업직, 해외 현지...","국제 경제ㆍ무역 관련 국가ㆍ민간 연구소, 인문과학 관련 국가ㆍ민간 연구소","외교관, 대사관, 인천국제공항공사, 한국수출입은행 등 영어 관련 공공기관"
2,중어중문학과,중어중문학과,520,인문계열,"글로벌언어문화학부(중국어중국학전공),글로벌커뮤니케이션학과(중화통상비즈니스전공),동아...","언론인(기자,PD,아나운서등), 여행안내원, 인문과학연구원, 작가, 출판물기획자, ...","관광통역안내사, 외국어번역행정사, 호텔경영사, 호텔관리사",정확 일치,"출판사, 무역 회사, 여행사, 호텔, 기업 일반 사무직 및 해외 영업직, 중국 현지...","국제 경제ㆍ무역 관련 국가ㆍ민간 연구소, 인문과학 관련 국가ㆍ민간 연구소","외교관, 대사관, 한국국제교류재단, 아시아문화원 등 중국어 관련 공공기관"
3,철학과,철학과,548,인문계열,"동양상담복지학과,동양철학과,미학과,아시아언어문화학부(철학전공),얼굴경영학과,역사·철...","언론인(기자,PD,아나운서등), 윤리교사, 인문과학연구원, 작가, 출판물기획전문가,...",중등학교2급정교사(철학),정확 일치,"언론사, 금융사, 문화콘텐츠 기획 및 경영 회사",철학 관련 연구소,"중앙정부 및 지방자치단체, 경제ㆍ인문사회연구회, 한국노동연구원 등 인문ㆍ사회과학 관..."
4,사학과,사학과,236,인문계열,"국사학과,국사학전공,동양사학과,미술사학과,미술사학전공,박물관학전공,사학,사학과,사학...","감정평가사, 국사ㆍ세계사교사, 언론인(기자,PD,아나운서등), 인문과학연구원, 인문...","문화재수리기술자, 박물관및미술관준학예사, 사서, 사서교사",정확 일치,"출판사, 초ㆍ중ㆍ고 및 대학 도서관, 기업의 문헌 자료실, 기업체의 사무직, 사설 ...","인문ㆍ사회과학 관련 국가ㆍ민간 연구소, 문화재 관련 연구소","중앙정부 및 지방자치단체의 문서실, 한국사학진흥재단, 국립박물관문화재단 등 역사학 ..."


### 엑셀로 저장하기

In [ ]:
#엑셀로 저장
#df__final.to_excel('엑셀저장명.xlsx', index=False)

NameError: name 'df__final' is not defined